# Implementation


In [1]:
%load_ext autoreload

In [2]:
%autoreload 2


In [3]:
import sys
sys.path.append('/home/ssm-user/Github/memento')

In [4]:
import scanpy as sc
import scipy.stats as stats
import scipy.sparse as sparse
import pandas as pd
import numpy as np
import string
import random
import logging

In [5]:
import memento

In [6]:
from memento import model

In [8]:
import memento.model.rna as rna
import memento.estimator.hypergeometric as hg

In [9]:
data_path = '/data_volume/memento/'

### Read some data

In [46]:
adata = sc.read(data_path + 'hbec/HBEC_type_I_filtered_counts_deep.h5ad')
adata.obs['q'] = 0.07

In [58]:
logging.basicConfig(
    format="%(asctime)s %(process)-7s %(levelname)-8s %(message)s",
    level=logging.INFO, 
    datefmt="%Y-%m-%d %H:%M:%S",
)
logging.captureWarnings(True)

In [61]:
rna.MementoRNA.setup_anndata(
    adata=adata,
    q_column='q',
    label_columns=['donor', 'stim'])

2023-06-23 16:48:21 62124   INFO     setup_anndata: creating groups
2023-06-23 16:48:21 62124   INFO     setup_anndata: computing cell sizes


In [62]:
adata.layers['count'] = adata.X

In [63]:
adata = adata[adata.obs['stim'] == 'control']

In [64]:
sf = adata.obs['memento_size_factor'].values

In [65]:
adata.X

<2360x36601 sparse matrix of type '<class 'numpy.float32'>'
	with 8171073 stored elements in Compressed Sparse Row format>

In [70]:
row_weight = (1/sf).reshape([-1, 1])

In [107]:
model = rna.MementoRNA(adata=adata)

In [111]:
model.compute_estimate(estimator=['mean', 'sem'], gene_list=['STAT1', 'ACTB'])

In [113]:
model.estimates['sem']

,STAT1,ACTB
sg^d2513^control,2.091550e-11,1.919589e-09
sg^d2614^control,4.122761e-12,9.275180e-10


In [72]:
row_weight.shape

(2360, 1)

In [73]:
adata.X.shape

(2360, 36601)

### Generate multi-index Pandas DataFrame

In [ ]:
adata

AnnData object with n_obs × n_vars = 69958 × 36601
    obs: 'NUM.SNPS', 'BEST.GUESS', 'DROPLET.TYPE', 'batch', 'HTO_classification', 'condition', 'donor', 'stim', 'time', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'